# Coummunity Detection


Visualize the graph community for diverse time.

[weekdays]<br>
1. commuted time<br>
    6-10<br>
    16-21<br>
2. regular time(except commuted time)<br>
    10-16<br>
    
[weekends]<br>

3. time on weekends<br>
    6-10<br>
    10-16<br>
    16-21



In [1]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import datetime as dt


import community as community_louvain

import folium
import matplotlib.cm as cm
import matplotlib.pyplot as plt


## Location EDA

- sensor location visualization
- community detection based on location

In [14]:
location = pd.read_csv('graph_sensor_locations_bay.csv',header = None)
location.columns=['sensor_id','latitude','longitude']
location

,sensor_id,latitude,longitude
0,400001,37.364085,-121.901149
1,400017,37.253303,-121.945440
2,400030,37.359087,-121.906538
3,400040,37.294949,-121.873109
4,400045,37.363402,-121.902233
...,...,...,...
320,413845,37.422887,-121.925747
321,413877,37.321613,-121.899642
322,413878,37.324641,-121.888603
323,414284,37.323066,-121.896538


In [25]:
map_osm = folium.Map(location=[37.364085,-121.901149], zoom_start=17)


for i in location.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = location.loc[i, 'sensor_id']
    lat = location.loc[i, 'latitude']
    lng = location.loc[i, 'longitude']
    
    # 추출한 정보를 지도에 표시
    marker = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color='blue'))
    marker.add_to(map_osm)


map_osm

In [83]:
### Read adj_matrix (DCRNN>data)
with open('./adj_mx_bay.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    adj_bay = u.load()
adj_bay = np.array(adj_bay)[-1]
adj_bay

G = nx.Graph(adj_bay)
# compute the best partition
partition = community_louvain.best_partition(G)


'''# draw the graph
pos = nx.spring_layout(G)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()


'''
set(partition.values()) ## 25 communities

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25}

In [44]:
colors = ['orange', 'blue', 'lightblue', 'lightred', 'beige', 'darkred', 'lightgreen', 'darkblue', 'black', 'darkgreen', 'pink', 'cadetblue', 'white', 'green', 'gray', 'darkpurple', 'purple', 'lightgray', 'red']
len(colors)

19

In [49]:


map_osm = folium.Map(location=[37.364085,-121.901149], zoom_start=17)


for i in location.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = location.loc[i, 'sensor_id']
    lat = location.loc[i, 'latitude']
    lng = location.loc[i, 'longitude']
    c = partition[i]
    # 추출한 정보를 지도에 표시 # coummunity 25개 찾았는데 색은 19개 뿐...ㅋㅋ
    if c <= 18:
        marker = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color=colors[c]))
        marker.add_to(map_osm)


map_osm

## Comumunity detection by Speed

In [4]:
'''
### Read speed data to do prediction task
speed_bay = pd.read_hdf('./pems-bay.h5')
speed_bay

'''


"\n### Read speed data to do prediction task\nspeed_bay = pd.read_hdf('./pems-bay.h5')\nspeed_bay\n\n"

In [50]:

speed_bay = pd.read_csv("bay.csv")
speed_bay

,Unnamed: 0,400001,400017,400030,400040,400045,400052,400057,400059,400065,...,409525,409526,409528,409529,413026,413845,413877,413878,414284,414694
0,2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,...,68.8,67.9,68.8,68.0,69.2,68.9,70.4,68.8,71.1,68.0
1,2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,...,68.4,67.3,68.4,67.6,70.4,68.8,70.1,68.4,70.8,67.4
2,2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,...,68.4,67.4,68.4,67.5,70.2,68.3,69.8,68.4,70.5,67.9
3,2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,...,68.5,67.5,68.5,67.5,70.4,68.7,70.2,68.4,70.8,67.6
4,2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,...,68.5,67.7,68.5,67.4,69.6,69.1,70.0,68.4,71.0,67.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52111,2017-06-30 23:35:00,70.9,66.3,68.4,61.0,61.8,66.9,66.0,67.4,65.4,...,64.6,66.8,64.7,61.1,68.4,61.4,70.5,68.2,71.6,66.2
52112,2017-06-30 23:40:00,71.3,66.6,68.7,60.9,62.0,66.2,66.5,68.2,64.9,...,65.1,67.3,64.8,60.8,69.8,62.2,69.4,68.6,71.6,68.4
52113,2017-06-30 23:45:00,71.4,66.9,68.1,61.1,62.0,66.7,65.8,68.2,64.2,...,64.9,66.5,64.6,61.3,69.5,63.8,70.6,68.4,71.6,66.6
52114,2017-06-30 23:50:00,72.2,66.5,68.0,61.1,62.5,65.8,67.1,68.3,64.0,...,64.1,65.9,64.8,61.3,69.6,64.2,70.2,68.7,71.6,68.4


### add day_name

In [51]:
speed_bay['Unnamed: 0'] = pd.to_datetime(speed_bay['Unnamed: 0'])
speed_bay['w_name'] = speed_bay['Unnamed: 0'].dt.day_name()
speed_bay.head()

,Unnamed: 0,400001,400017,400030,400040,400045,400052,400057,400059,400065,...,409526,409528,409529,413026,413845,413877,413878,414284,414694,w_name
0,2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,...,67.9,68.8,68.0,69.2,68.9,70.4,68.8,71.1,68.0,Sunday
1,2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,...,67.3,68.4,67.6,70.4,68.8,70.1,68.4,70.8,67.4,Sunday
2,2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,...,67.4,68.4,67.5,70.2,68.3,69.8,68.4,70.5,67.9,Sunday
3,2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,...,67.5,68.5,67.5,70.4,68.7,70.2,68.4,70.8,67.6,Sunday
4,2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,...,67.7,68.5,67.4,69.6,69.1,70.0,68.4,71.0,67.9,Sunday


### seperate data by time

6-10,10-16,16-21

In [52]:
date_time = list(speed_bay['Unnamed: 0'])

In [53]:
six_to_10 = []
ten_to_16 =[]
sixteens_to_21 = []

for i in range(len(date_time)):
    if date_time[i].hour in range(6,10):
        six_to_10.append(speed_bay.iloc[i])
        
    elif speed_bay['Unnamed: 0'][i].hour in range(10,16):
        ten_to_16.append(speed_bay.iloc[i])
        
    elif speed_bay['Unnamed: 0'][i].hour in range(16,21):
        sixteens_to_21.append(speed_bay.iloc[i])
        
        
six_to_10 = pd.DataFrame(six_to_10)
ten_to_16 = pd.DataFrame(ten_to_16)
sixteens_to_21 = pd.DataFrame(sixteens_to_21)

### weekdays and weekends

In [77]:
six_to_10_S = six_to_10[(six_to_10['w_name'] == 'Sunday')| (six_to_10['w_name'] == 'Saturday')]
six_to_10_W = six_to_10[(six_to_10['w_name']!= 'Sunday') & (six_to_10['w_name'] != 'Saturday')]


ten_to_16_S = ten_to_16[(ten_to_16['w_name'] == 'Sunday')| (ten_to_16['w_name'] == 'Saturday')]
ten_to_16_W = ten_to_16[(ten_to_16['w_name']!= 'Sunday') & (ten_to_16['w_name'] != 'Saturday')]

sixteens_to_21_S = sixteens_to_21[(sixteens_to_21['w_name'] == 'Sunday')| (sixteens_to_21['w_name'] == 'Saturday')]
sixteens_to_21_W = sixteens_to_21[(sixteens_to_21['w_name']!= 'Sunday') & (sixteens_to_21['w_name'] != 'Saturday')]


In [78]:
six_to_10_W

,Unnamed: 0,400001,400017,400030,400040,400045,400052,400057,400059,400065,...,409526,409528,409529,413026,413845,413877,413878,414284,414694,w_name
360,2017-01-02 06:00:00,67.0,68.2,68.6,67.1,67.9,65.9,68.4,67.3,65.0,...,64.6,67.8,66.6,68.2,55.9,71.7,67.7,70.0,67.1,Monday
361,2017-01-02 06:05:00,64.9,67.7,68.1,68.2,67.1,64.8,67.0,66.9,65.7,...,66.2,67.1,66.1,67.8,63.5,71.8,67.0,68.9,67.0,Monday
362,2017-01-02 06:10:00,64.9,67.8,67.4,67.7,66.7,62.6,67.1,66.5,65.9,...,66.9,66.8,65.7,66.5,65.3,71.1,66.7,68.2,65.7,Monday
363,2017-01-02 06:15:00,64.7,67.6,67.5,67.8,66.5,61.3,66.7,66.4,65.6,...,66.3,66.6,65.4,66.5,66.7,69.9,66.5,68.0,64.2,Monday
364,2017-01-02 06:20:00,64.1,67.4,67.4,67.5,66.2,58.6,65.2,66.4,64.7,...,66.1,66.4,65.3,66.3,65.7,69.8,66.3,66.3,63.5,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51943,2017-06-30 09:35:00,62.7,64.9,59.0,55.5,35.8,63.0,53.0,66.7,58.4,...,62.7,59.7,56.2,62.4,30.8,72.1,62.6,67.5,61.3,Friday
51944,2017-06-30 09:40:00,63.2,65.3,58.2,55.7,37.7,64.1,57.5,66.2,60.4,...,61.9,59.7,56.5,63.1,32.7,74.2,61.5,67.4,61.4,Friday
51945,2017-06-30 09:45:00,62.0,65.5,56.5,56.3,47.6,65.0,59.4,64.5,61.7,...,63.1,59.9,56.4,62.5,31.5,74.3,62.1,67.6,62.9,Friday
51946,2017-06-30 09:50:00,61.4,64.9,56.9,55.0,52.0,64.5,58.1,66.4,61.5,...,64.2,60.2,57.2,62.0,31.1,72.9,62.4,67.5,63.8,Friday


### make adjacent matrix

- for mean of sensor's speed
- by using gaussian kernels

In [79]:
def Gaussian(x,z,sigma):
    return np.exp((-(np.linalg.norm(x-z)**2))/(2*sigma**2))

def GaussianMatrix(X,sigma):
    row,col=X.shape
    GassMatrix=np.zeros(shape=(row,row))
    X=np.asarray(X)
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            GassMatrix[i,j]=Gaussian(v_i.T,v_j.T,sigma)
            j+=1
        i+=1
    return GassMatrix

def adj_mat_speed(data):
    mean = pd.DataFrame(data.mean(axis = 0))
    adj_mat = GaussianMatrix(mean,sigma = 1)
    
    return adj_mat

In [95]:
def visualization_community(data,location):
    
    G = nx.Graph(adj_mat_speed(data))
    # compute the best partition
    partition = community_louvain.best_partition(G)
    map_osm = folium.Map(location=[37.364085,-121.901149], zoom_start=12)

    for i in location.index:
        # 행 우선 접근 방식으로 값 추출하기
        name = location.loc[i, 'sensor_id']
        lat = location.loc[i, 'latitude']
        lng = location.loc[i, 'longitude']
        c = partition[i]
        
        # can visualize 18th community.(because of limiation of Marker)
        if c <= 18:
            marker = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color=colors[c]))
            marker.add_to(map_osm)

    return map_osm

### 6-10 weekdays

In [96]:
visualization_community(six_to_10_W,location)

<ipython-input-79-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))


### 6-10 weekends

In [97]:
visualization_community(six_to_10_S,location)

<ipython-input-79-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))


### 6-10 all days

In [98]:
visualization_community(six_to_10,location)

<ipython-input-79-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
